# Experiment with Pytorch
to create endocder decoder ocr model with cnn and lstm

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Setup path in .env file
import os
from dotenv import load_dotenv
import cv2
import numpy as np
import torch

# Get absolut path to proeject root
load_dotenv()
project_root_dir = os.path.dirname(os.path.abspath("./"))
print("Project root dir:", project_root_dir)

data_dir = os.getenv("PUBTABNET_CROPPED_DATA_DIR")
absolute_dir = project_root_dir + data_dir
print("Absolute path:", absolute_dir)

Project root dir: /Users/leonremke/Documents/GIT_REPOS/UNI/neural_networks_seminar
Absolute path: /Users/leonremke/Documents/GIT_REPOS/UNI/neural_networks_seminar/PubTabNet_cropped


In [3]:
image_dir_train = f"{absolute_dir}/train"
image_dir_val = f"{absolute_dir}/val"
label_file_train = f"{absolute_dir}/PubTabNet_2.0.0_train_separated.json"
label_file_val = f"{absolute_dir}/PubTabNet_2.0.0_val_separated.json"

In [4]:
import sys
sys.path.append('../')
from torch.utils.data import DataLoader
label_file_small = f"{absolute_dir}/subset_small.json"


In [5]:
# label_file_train = f"{absolute_dir}/PubTabNet_2.0.0_train.json"
# label_file_val = f"{absolute_dir}/PubTabNet_2.0.0_val.json"

# # Function to generate labels
# import json
# def generate_labels(json_data):
#     labels = []
#     for key, value in json_data.items():
#         cells = value["html"]["cells"]
#         for i, cell in enumerate(cells):
#             bbox = cell.get('bbox')
#             tokens = cell.get('tokens')
#             if bbox is None:
#                 continue
#             label = key.replace(".png", f"_bbox_{bbox[0]}_{bbox[1]}_{bbox[2]}_{bbox[3]}.png")
#             label = {
#                 "filename": key.replace(".png", f"_bbox_{bbox[0]}_{bbox[1]}_{bbox[2]}_{bbox[3]}.png"),
#                 "split": value["split"],
#                 "imgid": value["imgid"],
#                 "tokens": tokens,
#                 "bbox": bbox,
#             }
#             labels.append(label)
#     return labels

# # Generate labels
# with open(label_file_val, 'r') as f:
#         labels = json.load(f)
#         result = generate_labels(labels)

# # Specify the output file name
# output_file_name = f"{absolute_dir}/PubTabNet_2.0.0_val_separated.json"

# # Write the generated labels to a new JSON file
# with open(output_file_name, 'w') as output_file:
#     json.dump(result, output_file, indent=4)

# print(f"Generated labels have been written to {output_file_name}")

In [6]:
from torchvision import transforms
transform_list =  [transforms.Grayscale(1),
                            transforms.ToTensor(), 
                            transforms.Normalize((0.5,), (0.5,))]
transform = transforms.Compose(transform_list)

In [7]:
# Example usage:
from custom_ocr_cnn_lstm.dataset import OCRDataset
from custom_ocr_cnn_lstm.dataset_cropped import OCRDatasetCropped

train_dataset = OCRDatasetCropped(label_file_train, image_dir_train, transform=transform)
val_dataset = OCRDatasetCropped(label_file_val, image_dir_val, transform=transform)

# Accessing a sample
sample = train_dataset[0]
print(sample)

{'idx': 0, 'label': 'Species', 'image': tensor([[[ 1.0000,  0.9451,  0.5765,  0.7569,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  0.9294,  0.8902,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 1.0000,  0.0196,  0.9686,  0.5922,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  0.8824,  0.8353,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 1.0000, -0.2941,  0.8353,  1.0000,  0.8980,  0.5765,  0.3961,
           0.7412,  1.0000,  0.8039,  0.5529,  0.9608,  0.9294,  0.4745,
           0.6392,  0.7490,  0.7804,  1.0000,  0.7020,  0.6078,  1.0000,
           0.7020,  0.5608,  1.0000],
         [ 1.0000,  0.6235, -0.3804,  0.4588,  0.7804, -0.2000,  0.6157,
          -0.2706,  0.8980,  0.0118,  0.3490,  0.3725,  0.1765,  0.9765,
           0.6863,  0.4275,  0.4039,  0.560

In [8]:
max_height = 0
max_width = 0

for item in train_dataset:
    image = item['image']
    height, width = image.shape[1], image.shape[2]
    max_height = max(max_height, height)
    max_width = max(max_width, width)

print(f"Maximum Height: {max_height}")
print(f"Maximum Width: {max_width}")


Maximum Height: 84
Maximum Width: 429


In [9]:
# class SynthCollator(object):
    
#     def __call__(self, batch):
#         width = [item['image'].shape[2] for item in batch]
#         indexes = [item['idx'] for item in batch]
#         imgs = torch.ones([len(batch), batch[0]['image'].shape[0], batch[0]['image'].shape[1], 
#                            max(width)], dtype=torch.float32)
#         for idx, item in enumerate(batch):
#             try:
#                 imgs[idx, :, :, 0:item['image'].shape[2]] = item['image']
#             except:
#                 continue
#                 #print(imgs.shape)
#         item = {'image': imgs, 'idx':indexes}
#         if 'label' in batch[0].keys():
#             labels = [item['label'] for item in batch]
#             item['label'] = labels
#         return item

import torch
import torch.nn.functional as F

class CustomCollator(object):
    
    def __init__(self, target_height = max_height, target_width= max_width):
        self.target_height = target_height
        self.target_width = target_width
    
    def __call__(self, batch):
        indexes = [item['idx'] for item in batch]
        
        # Resize images to the target size
        resized_images = []
        for item in batch:
            image = item['image']
            resized_image = F.interpolate(image.unsqueeze(0), size=(self.target_height, self.target_width), mode='bilinear', align_corners=False)
            resized_images.append(resized_image.squeeze(0))
        
        # Stack resized images
        imgs = torch.stack(resized_images, dim=0)
        
        item = {'image': imgs, 'idx': indexes}
        
        if 'label' in batch[0].keys():
            labels = [item['label'] for item in batch]
            item['label'] = labels
        
        return item


In [10]:
# Example usage:
from custom_ocr_cnn_lstm.dataset import OCRDataset
from custom_ocr_cnn_lstm.dataset_cropped import OCRDatasetCropped

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn=CustomCollator())
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True, collate_fn=CustomCollator())

In [11]:
# Vielleicht liegt der Fehler hier. 
# Batch enthält jeweils 3 items die dann 10 elemente jeweils enthalten
for batch_idx, samples in enumerate(train_loader):
    print("------")
    print(samples["image"].shape)
    print(batch_idx, samples["image"][0], samples["label"][0])
    break

------
torch.Size([10, 1, 84, 429])
0 tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]]]) 33.82 ± 0.76


In [12]:
import re
def remove_xml_tags(input_string):
    pattern = r'<[^>]+>'
    return re.sub(pattern, '', input_string)

In [13]:
import json
def create_alphabet_file(alphabet, file_path):
    with open(file_path, 'w') as f:
        # f.write("START\n")
        for char in alphabet:
            f.write(char + "\n")
        # f.write("END\n")

# Replace these with your actual ground truth labels
train_labels = train_dataset.labels
val_labels = train_dataset.labels
# Concat both dicts
labels = train_labels + val_labels

alph_labels = []
for label in labels:
    label_tokens = label.get('tokens')
    label = " ".join(" ".join(tokens) for tokens in label_tokens)
    label = remove_xml_tags(label)
    alph_labels.append(label)
print("Ground truth labels loaded: ", len(alph_labels), alph_labels[:100])

unique_chars = set()
unique_radicals = set()

# Loop through ground truth labels to extract unique characters and radicals
for label in alph_labels:
    for char in label:
        unique_chars.add(char)
        # You might need to extract radicals from each character here if using decomposition

# Define the file paths for character and radical alphabets
char_alphabet_file_path = f"{absolute_dir}/character_alphabet.txt"
radical_alphabet_file_path = f"{absolute_dir}/radical_alphabet.txt"

# Create character alphabet file
create_alphabet_file(unique_chars, char_alphabet_file_path)
print(f"Character alphabet file created at: {char_alphabet_file_path}")

# Create radical alphabet file (if needed)
# create_alphabet_file(unique_radicals, radical_alphabet_file_path)
# print(f"Radical alphabet file created at: {radical_alphabet_file_path}")

Ground truth labels loaded:  13690 [' S p e c i e s ', ' A n a j a ́ s ', ' P o r t e l ', ' S S B V ', ' W i l d ', ' R u r a l ', ' U r b a n ', ' W i l d ', ' R u r a l ', ' U r b a n ', ' W i l d ', ' R u r a l ', ' U r b a n ', ' T o t a l ', ' ( % ) ', ' E v a n d r o m y i a   w a l k e r i ', '4 4', '4 0', '1', '1 5 6', '2 9 6', '1', '5 1', '1', '2', '5 9 2', '6 8 . 8 4', ' E v a n d r o m y i a   i n f r a s p i n o s a ', '4 4', '0', '3', '8 2', '1', '0', '0', '0', '0', '1 3 0', '1 5 . 1 2', ' N y s s o m y i a   a n t u n e s i  a  ', '1 1', '3', '3', '2 0', '3', '0', '1', '0', '0', '4 1', '4 . 7 7', ' M i c r o p y g o m y i a   r o r o t a e n s i s ', '2 0', '1', '0', '4', '0', '0', '2', '0', '0', '2 7', '3 . 1 4', ' S c i o p e m y i a   s o r d e l l i i ', '7', '1', '0', '1 3', '2', '0', '2', '0', '0', '2 5', '2 . 9 1', ' B i c h r o m o m y i a   f l a v i s c u t e l l a t a  a  ', '0', '0', '0', '4', '0', '0', '1 6', '0', '0', '2 0', '2 . 3 2', ' N y s s o m y i a  

https://deepayan137.github.io/blog/markdown/2020/08/29/building-ocr.html#the-ctc-loss

In [14]:
# From https://deepayan137.github.io/blog/markdown/2020/08/29/building-ocr.html#the-ctc-loss
class CustomCTCLoss(torch.nn.Module):
    # T x B x H => Softmax on dimension 2
    def __init__(self, dim=2):
        super().__init__()
        self.dim = dim
        self.ctc_loss = torch.nn.CTCLoss(reduction='mean', zero_infinity=True)

    def forward(self, logits, labels,
            prediction_sizes, target_sizes):
        EPS = 1e-7
        loss = self.ctc_loss(logits, labels, prediction_sizes, target_sizes)
        loss = self.sanitize(loss)
        return self.debug(loss, logits, labels, prediction_sizes, target_sizes)
    
    def sanitize(self, loss):
        EPS = 1e-7
        if abs(loss.item() - float('inf')) < EPS:
            return torch.zeros_like(loss)
        if math.isnan(loss.item()):
            return torch.zeros_like(loss)
        return loss

    def debug(self, loss, logits, labels,
            prediction_sizes, target_sizes):
        if math.isnan(loss.item()):
            print("Loss:", loss)
            print("logits:", logits)
            print("labels:", labels)
            print("prediction_sizes:", prediction_sizes)
            print("target_sizes:", target_sizes)
            raise Exception("NaN loss obtained. But why?")
        return loss

In [15]:
# Load alpabet from file
def load_alphabet(file_path):
    with open(file_path, 'r') as f:
        alphabet = f.read().splitlines()
    return alphabet

alphabet = load_alphabet(char_alphabet_file_path)
print("Alphabet loaded:", len(alphabet), alphabet)

Alphabet loaded: 111 ['<', 'I', '&', 'O', 'Q', '‐', 'G', '–', '#', 'p', '“', 'A', 'P', 'l', 'D', '§', '≤', '⁄', '·', '+', '`', 'α', '.', ' ', ',', 'B', 'Δ', 'a', '7', '>', '≥', 'k', ']', '°', '6', '0', 'U', 'W', '%', '8', 'L', 'w', 'i', 'F', 'V', 'n', 'y', 'o', '=', 'd', 'H', 'z', 'v', '$', '[', 'S', '́', 'q', 'T', 'Y', '5', '(', 'c', 'j', 'J', 'b', 'x', 'Z', '′', 'E', '×', 'C', 't', '—', 'K', 'f', 's', '1', 'γ', '4', '”', '*', 'g', '’', '†', '/', '2', ')', 'u', "'", 'X', 'M', 'R', ':', 'e', '_', 'μ', 'λ', '→', 'h', '±', 'N', ';', '?', 'β', '9', 'r', '−', '-', '3', 'm']


In [16]:
def find_maximum_width(dataset):
    # Initialize a list to store image widths
    image_widths = []

    # Iterate through the dataset and collect image widths
    for sample in dataset:
        image_width = sample['image'].shape[2]  # Get the width of the image
        image_widths.append(image_width)

    # Find the maximum width across all images
    max_width = max(image_widths)

    return max_width

# Example usage:
max_width_train = find_maximum_width(train_dataset)
max_width_val = find_maximum_width(val_dataset)

max_width = max(max_width_train, max_width_val)

print("Maximum width in dataset:", max_width)


Maximum width in dataset: 454


In [17]:
# Setup environment for training
import numpy as np
from collections import OrderedDict
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn.utils.clip_grad import clip_grad_norm_
from utils import OCRLabelConverter
from tqdm import *
from cnn_lstm_ocr import CNNLSTM_OCR

# params = {
#     "input_dim": 256,
#     "hidden_dim": 256,
#     "output_dim": 1,
#     "input_planes": 1,
#     "planes": 1,
#     "schedule": False,
#     'image_height':32,
#     'number_channels':1,
#     'number_hidden_layers':256,
#     'len_alphabet':len(alphabet),
#     'learning_rate':0.001,
#     'epochs':4,
#     'batch_size':10,
#     'model_dir':'model_history',
#     'log_dir':'logs',
#     'resume':False,
#     'cuda':False,
#     'schedule':False    
# }


params = {
    "input_dim": 1,  # Assuming grayscale images
    "alphabet": alphabet,
    "hidden_dim": 256,
    "output_dim": len(alphabet) + 1,  # Number of classes (including a blank symbol)
    "input_planes": 1,  # Assuming grayscale images
    "planes": 32,  # Adjust this value as needed
    'image_height': 32,  # Adjust as needed
    'number_channels': 1,  # Assuming grayscale images
    'number_hidden_layers': 2,  # Adjust as needed
    'len_alphabet': len(alphabet),  # Number of classes (excluding a blank symbol)
    'learning_rate': 0.001,
    'epochs': 1,
    'batch_size': 10,
    'model_dir': 'model_history',
    'log_dir': 'logs',
    'resume': False,
    'cuda': False,
    'schedule': False,
    'max_width': max_width
}



model = CNNLSTM_OCR(params)
converter = OCRLabelConverter(''.join(params['alphabet']))
criterion = CustomCTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = CosineAnnealingLR(optimizer, T_max=params['epochs'])
batch_size = params['batch_size']
count = 1
epochs = params['epochs']
cuda = params['cuda']
print("Params:", params)

def train_dataloader():
        # logging.info('training data loader called')
        loader = torch.utils.data.DataLoader(self.data_train,
                batch_size=self.batch_size,
                collate_fn=self.collate_fn,
                shuffle=True)
        return loader

Params: {'input_dim': 32, 'alphabet': ['<', 'I', '&', 'O', 'Q', '‐', 'G', '–', '#', 'p', '“', 'A', 'P', 'l', 'D', '§', '≤', '⁄', '·', '+', '`', 'α', '.', ' ', ',', 'B', 'Δ', 'a', '7', '>', '≥', 'k', ']', '°', '6', '0', 'U', 'W', '%', '8', 'L', 'w', 'i', 'F', 'V', 'n', 'y', 'o', '=', 'd', 'H', 'z', 'v', '$', '[', 'S', '́', 'q', 'T', 'Y', '5', '(', 'c', 'j', 'J', 'b', 'x', 'Z', '′', 'E', '×', 'C', 't', '—', 'K', 'f', 's', '1', 'γ', '4', '”', '*', 'g', '’', '†', '/', '2', ')', 'u', "'", 'X', 'M', 'R', ':', 'e', '_', 'μ', 'λ', '→', 'h', '±', 'N', ';', '?', 'β', '9', 'r', '−', '-', '3', 'm'], 'hidden_dim': 256, 'output_dim': 112, 'input_planes': 1, 'planes': 32, 'image_height': 32, 'number_channels': 1, 'number_hidden_layers': 2, 'len_alphabet': 111, 'learning_rate': 0.001, 'epochs': 1, 'batch_size': 10, 'model_dir': 'model_history', 'log_dir': 'logs', 'resume': False, 'cuda': False, 'schedule': False, 'max_width': 454}


In [18]:
# import torch
# import torch.nn as nn
# from torchvision import transforms
# from torch.optim.lr_scheduler import CosineAnnealingLR
# from tqdm import tqdm
# from custom_ocr_cnn_lstm.dataset_cropped import OCRDatasetCropped

# # Define a method for training the OCR model
# def train_ocr_model(model, train_loader, optimizer, criterion, device, epoch, epochs):
#     model.train()
#     total_loss = 0.0

#     progress_bar = tqdm(train_loader, desc='Epoch: [%d]/[%d] Training'%(epoch, 
#                 epochs), leave=True)

#     for idx, batch in enumerate(progress_bar):
#         optimizer.zero_grad()
#         images, labels = batch['image'], batch['label']
        
#         # Forward pass
#         outputs = model(images)
        
#         # Calculate the loss
#         loss = criterion(outputs, labels)
        
#         # Backpropagation
#         loss.backward()
#         clip_grad_norm_(model.parameters(), max_norm=5)  # Clip gradients to prevent exploding gradients
        
#         optimizer.step()
#         total_loss += loss.item()
    
#     avg_loss = total_loss / len(train_loader)
#     return avg_loss

# # Define a method for evaluating the OCR model on the validation dataset
# def evaluate_ocr_model(model, val_loader, criterion, device):
#     model.eval()
#     total_loss = 0.0

#     with torch.no_grad():
#         for images, labels in tqdm(val_loader, desc="Validation"):
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             total_loss += loss.item()
    
#     avg_loss = total_loss / len(val_loader)
#     return avg_loss

# # Define a method for OCR inference
# def ocr_inference(model, image, transform, device):
#     model.eval()
#     with torch.no_grad():
#         # Preprocess the input image using the same transform as during training
#         image = transform(image).unsqueeze(0).to(device)
#         outputs = model(image)
        
#         # Perform any necessary post-processing on the model's outputs to get the OCR result
#         # You may need to implement this part based on your specific OCR task
    
#     return ocr_result


# # Set device (CPU or GPU) for training
# device = torch.device("cuda" if params['cuda'] and torch.cuda.is_available() else "cpu")
# model.to(device)

# # Training loop
# for epoch in range(params['epochs']):
#     print(f"Epoch {epoch + 1}/{params['epochs']}")
    
#     # Train the model
#     train_loss = train_ocr_model(model, train_loader, optimizer, criterion, device, epoch, params['epochs'])
#     print(f"Train Loss: {train_loss:.4f}")
    
#     # Validate the model
#     val_loss = evaluate_ocr_model(model, val_loader, criterion, device)
#     print(f"Validation Loss: {val_loss:.4f}")

# # After training, you can use the trained model for inference on new images
# # Example usage for OCR inference:
# sample_image, sample_label = train_dataset[0]  # Load a sample image and label
# ocr_result = ocr_inference(model, sample_image, transform, device)
# print("OCR Result:", ocr_result)


In [19]:
# # Define a mapping from characters to integers
# char_to_int = {}
# int_to_char = {}
# with open(char_alphabet_file_path, 'r') as f:
#     alphabet_lines = f.read().splitlines()

# for i, char in enumerate(alphabet_lines[1:-1]):  # Skip "START" and "END" lines
#     char_to_int[char] = i
#     int_to_char[i] = char

# # Define a special key for space character
# SPACE_INT = len(char_to_int)  # Use the last index in the alphabet

# # Implement text_to_tensor method
# def text_to_tensor(text):
#     # Convert text to a list of integers using the char_to_int mapping
#     tensor = [char_to_int[char] if char in char_to_int else SPACE_INT for char in text]
#     length = [len(text)]
#     # return tensor
#     return (torch.IntTensor(tensor), torch.IntTensor(length))

# # Implement tensor_to_text method
# def tensor_to_text(tensor):
#     # Convert a tensor (list of integers) back to text using the int_to_char mapping
#     text = "".join([int_to_char[idx] if idx != SPACE_INT else ' ' for idx in tensor])
#     return text

# # Example usage:
# text = "HELLO 5 7"
# tensor = text_to_tensor(text)
# print("Text to Tensor:", tensor)
# text_reconstructed = tensor_to_text(tensor)
# print("Tensor to Text:", text_reconstructed)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from custom_ocr_cnn_lstm.cnn_lstm_ocr import CNNLSTM_OCR
import math

# Define your model, criterion, optimizer, and other parameters here

# Initialize your model, criterion, and optimizer
model = CNNLSTM_OCR(params)  # Assuming you've already defined CNNLSTM_OCR class
# criterion = nn.CTCLoss()
criterion = CustomCTCLoss()
optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = CosineAnnealingLR(optimizer, T_max=params['epochs'])

# torch.autograd.set_detect_anomaly(True)

# Define training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0

    for batch in tqdm(dataloader, desc="Training"):
        inputs = batch['image'].to(device)
        targets = batch['label']

        targets_encoded = []
        targets_encoded_length = []
        for target in targets: 
            target, length = converter.encode(target)
            targets_encoded.append(target)
            targets_encoded_length.append(len(length))
        
        # Find the length of the longest tensor
        max_len = max(len(t) for t in targets_encoded)

        # Pad tensors to the length of the longest tensor
        padded_tensors = [F.pad(t, (0, max_len - len(t)), value=112) for t in targets_encoded]

        # Stack the padded tensors
        targets_encoded = torch.stack(padded_tensors)

        # target lengths as tensor
        targets_encoded_length = torch.IntTensor(targets_encoded_length)

        #print("#"*50)
        #print(f"Input shape: {inputs.shape}")
        #print(f"size targets: {targets_encoded.shape}")

        optimizer.zero_grad()
        # print("cnn input inputs.shape:", inputs.shape)
        outputs = model(inputs)
        outputs = outputs.contiguous().cpu()
        outputs = torch.nn.functional.log_softmax(outputs, 2)
        # b is batch size, T is input sequence and h is hidden size
        b, T, h = outputs.size()
        #print("outputs shape:", outputs.shape)
        #print(outputs.transpose(0, 1).shape)
        # print("targets shape:", targets.size())
        pred_sizes = torch.LongTensor([T for i in range(b)])
        #print("pred_sizes shape:", pred_sizes.shape)
        # Calculate the CTC loss
        # output_lengths = torch.full(size=(inputs.size(0),), fill_value=outputs.size(0), dtype=torch.long)
        #print(outputs.shape, targets.shape, pred_sizes, lengths)
        #print(len(pred_sizes), len(lengths), len(targets), len(outputs))
        loss = criterion(outputs.transpose(0, 1), targets_encoded, pred_sizes, targets_encoded_length)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    return total_loss / len(dataloader)

# Define validation method
# def validate(model, dataloader, criterion, device):
#     model.eval()
#     total_loss = 0.0

#     with torch.no_grad():
#         for batch in tqdm(dataloader, desc="Validation"):
#             inputs = batch['image'].to(device)
#             targets = batch['label']
#             # Assuming targets need to be converted to tensor, e.g., using a function `text_to_tensor`
#             targets = text_to_tensor(targets) 

#             outputs = model(inputs)
            
#             # Calculate the CTC loss
#             output_lengths = torch.full(size=(inputs.size(0),), fill_value=outputs.size(0), dtype=torch.long)
#             loss = criterion(outputs, targets, input_lengths=None, target_lengths=output_lengths)
            
#             total_loss += loss.item()

#     return total_loss / len(dataloader)

# Training loop
device = torch.device("cuda" if params['cuda'] and torch.cuda.is_available() else "cpu")
model.to(device)
train_dataloader = train_loader  # Assuming you have train_loader defined
val_dataloader = val_loader  # Assuming you have val_loader defined

for epoch in range(params['epochs']):
    train_loss = train(model, train_dataloader, criterion, optimizer, device)
    # val_loss = validate(model, val_dataloader, criterion, device)
    scheduler.step()  # Adjust learning rate
    
    print(f"Epoch {epoch+1}/{params['epochs']} - Train Loss: {train_loss:.4f} - Validation Loss: {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), params['model_dir'] + '/model.pth')


Training: 100%|██████████| 685/685 [26:05<00:00,  2.28s/it]


NameError: name 'val_loss' is not defined

In [23]:
torch.save(model.state_dict(), params['model_dir'] + '/model.pth')

In [22]:
!pip install textdistance

In [ ]:
def validate(model, dataloader, criterion, converter, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            inputs = batch['image'].to(device)
            targets = batch['label']

            targets_encoded = []
            targets_encoded_length = []
            for target in targets: 
                target, length = converter.encode(target)
                targets_encoded.append(target)
                targets_encoded_length.append(len(length))
            
            # Find the length of the longest tensor
            max_len = max(len(t) for t in targets_encoded) + 2
            
            # Pad tensors to the length of the longest tensor
            padded_tensors = [F.pad(t, (0, max_len - len(t)), value=0) for t in targets_encoded]

            # Stack the padded tensors
            # targets_encoded = torch.stack(padded_tensors)
            targets_encoded = torch.stack(padded_tensors)

            optimizer.zero_grad()
            # print("cnn input inputs.shape:", inputs.shape)
            outputs = model(inputs)
            outputs = outputs.contiguous().cpu()
            outputs = torch.nn.functional.log_softmax(outputs, 2)
            # b is batch size, T is input sequence and h is hidden size
            b, T, h = outputs.size()
            # print("outputs shape:", outputs.shape)
            # print(outputs.transpose(0, 1).shape)
            # print("targets shape:", targets.size())
            pred_sizes = torch.LongTensor([T for i in range(b)])
            pred_sizes = torch.LongTensor([T for i in range(b)]).to(device=device)

            print(outputs)

            #print("pred_sizes shape:", pred_sizes.shape)
            # Calculate the CTC loss
            # output_lengths = torch.full(size=(inputs.size(0),), fill_value=outputs.size(0), dtype=torch.long)
            #print(outputs.shape, targets.shape, pred_sizes, lengths)
            #print(len(pred_sizes), len(lengths), len(targets), len(outputs))
            total_loss += criterion(outputs.transpose(0, 1), targets_encoded, pred_sizes, targets_encoded_length)

    avg_loss = total_loss / len(dataloader)
    return avg_loss

agv_loss_training = validate(model, val_dataloader, criterion=criterion, converter=converter, device=device)
print(agv_loss_training)


In [48]:
from utils import Eval
import matplotlib as plt
def get_accuracy(loader, model, converter):
    model.eval()
    evaluator = Eval()
    labels, predictions, images = [], [], []
    for iteration, batch in enumerate(tqdm(loader)):
        input_, targets = batch['image'].to(device), batch['label']
        images.extend(input_.squeeze().detach())
        labels.extend(targets)
        targets, lengths = converter.encode(targets)
        logits = model(input_).transpose(1, 0)
        # print(logits)
        logits = torch.nn.functional.log_softmax(logits, 2)
        logits = logits.contiguous().cpu()
        T, B, H = logits.size()
        pred_sizes = torch.LongTensor([T for i in range(B)])
        probs, pos = logits.max(2)
        print(probs, pos)

        print(pos)
        pos = pos.transpose(1, 0).contiguous().view(-1)
        sim_preds = converter.decode(pos.data, pred_sizes.data, raw=False)
        print(sim_preds)
        predictions.extend(sim_preds)
        
#     make_grid(images[:10], nrow=2)
    fig=plt.figure(figsize=(8, 8))
    columns = 4
    rows = 5
    pairs = list(zip(images, predictions))
    indices = np.random.permutation(len(pairs))
    for i in range(1, columns*rows +1):
        img = images[indices[i]]
        img = (img - img.min())/(img.max() - img.min())
        img = np.array(img * 255.0, dtype=np.uint8)
        fig.add_subplot(rows, columns, i)
        plt.title(predictions[indices[i]])
        plt.axis('off')
        plt.imshow(img)
    plt.show()
    ca = np.mean((list(map(evaluator.char_accuracy, list(zip(predictions, labels))))))
    wa = np.mean((list(map(evaluator.word_accuracy_line, list(zip(predictions, labels))))))
    return ca, wa

In [49]:
model_state_dir_path = params['model_dir']+"/model.pth"
print(model_state_dir_path)
if os.path.isfile(model_state_dir_path):
    print('Loading model %s'%model_state_dir_path)
    state_dict = torch.load(model_state_dir_path)
    model = CNNLSTM_OCR(params=params)
    model.load_state_dict(state_dict=state_dict)
    ca, wa = get_accuracy(val_loader, model, converter)
    print("Character Accuracy: %.2f\nWord Accuracy: %.2f"%(ca, wa))
else:
    print('Exiting')

model_history/model.pth
Loading model model_history/model.pth


  0%|          | 0/613 [00:00<?, ?it/s]

  0%|          | 1/613 [00:00<05:54,  1.73it/s]

tensor([[-0.0499, -0.0509, -0.0498,  ..., -0.0503, -0.0498, -0.0498],
        [-0.0067, -0.0062, -0.0067,  ..., -0.0068, -0.0066, -0.0066],
        [-0.0015, -0.0016, -0.0015,  ..., -0.0014, -0.0015, -0.0015],
        ...,
        [-0.0033, -0.0064, -0.0033,  ..., -0.0033, -0.0033, -0.0033],
        [-0.0159, -0.0215, -0.0159,  ..., -0.0159, -0.0159, -0.0159],
        [-0.0629, -0.0620, -0.0629,  ..., -0.0629, -0.0629, -0.0629]],
       grad_fn=<MaxBackward0>) tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
['', '', '', '', '', '', '', '', '', '']


  0%|          | 2/613 [00:01<06:35,  1.55it/s]

...

tensor([[-0.0498, -0.0498, -0.0498,  ..., -0.0498, -0.0499, -0.1892],
        [-0.0066, -0.0067, -0.0066,  ..., -0.0066, -0.0067, -0.0628],
        [-0.0015, -0.0015, -0.0015,  ..., -0.0015, -0.0015, -0.0122],
        ...,
        [-0.0033, -0.0033, -0.0033,  ..., -0.0033, -0.0033, -0.0059],
        [-0.0159, -0.0159, -0.0159,  ..., -0.0159, -0.0159, -0.0250],
        [-0.0629, -0.0629, -0.0629,  ..., -0.0629, -0.0629, -0.0760]],
       grad_fn=<MaxBackward0>) tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
['', '', '', '', '', '', '', '', '', '']


  1%|          | 5/613 [00:03<06:17,  1.61it/s]

tensor([[-0.1892, -0.0498, -0.2021,  ..., -0.0499, -0.0498, -0.0498],
        [-0.0628, -0.0066, -0.0823,  ..., -0.0067, -0.0066, -0.0066],
        [-0.0122, -0.0015, -0.0192,  ..., -0.0015, -0.0015, -0.0015],
        ...,
        [-0.0059, -0.0033, -0.0033,  ..., -0.0033, -0.0033, -0.0033],
        [-0.0250, -0.0159, -0.0159,  ..., -0.0159, -0.0159, -0.0159],
        [-0.0760, -0.0629, -0.0629,  ..., -0.0629, -0.0629, -0.0629]],
       grad_fn=<MaxBackward0>) tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
['', '', '', '', '', '', '', '', '', '']


  1%|          | 5/613 [00:03<07:25,  1.37it/s]


KeyboardInterrupt: 